# Midterm

In [10]:
import numpy as np
import scipy as sci
import sympy as sym
import pandas as pd
import seaborn as sns
import matplotlib as mp
import matplotlib.pyplot as plt

from numpy import exp, pi
from scipy.integrate import quad
from scipy.optimize import curve_fit 
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
WC = pd.read_csv('/Users/kev/Documents/Python/MATH-377---Math-Modeling/Data Sets/whooping_crane.csv', 
                 header=1)

## Question 1: Plant population

In [3]:
# Probabilty 
def prob(P, a, r):
    s = P*r
    p = 1 - exp(-a/s)
    return p

# Population (simulated)
def Plant_pop(P0, a, n, r):
    Pop = np.zeros(n) # Population over time
    Pop[0] = P0 # Initial population
    for i in range(0, n-1):
        S = np.zeros(int(r*Pop[i])) # Largest possible amount of seeds produced
        for j in range(0, len(S)):
            p = prob(Pop[i], a, r) # Probability of seeds surviving
            S[j] = np.random.choice([0, 1], p=[1-p, p]) # Which seeds survive
            j += 1
        Seed = np.sum(S) # Total surviving seeds
        Pop[i+1] = Seed # Plant population next season 
        i += 1
    return Pop

# Popluation (theoretical)
def Plant_popT(P0, a, n, r):
    P =  np.zeros(n+1)
    P[0] += P0
    for i in range(0, n):
        s = P[i] * r
        P[i+1] += r * (1-exp(-a/s)) * P[i]
        i += 1
    return P

In [4]:
sims = 3
Plant_popt = Plant_popT(P0=1, a=100, n=50, r=2)

fig, ax = plt.subplots(1, 1, figsize=(10, 4))

for i in range(0, sims):
    Plant_popA = Plant_pop(P0=1, a=100, n=50, r=2)
    ax.plot(Plant_popA, label="Simulation {0}".format(i))
    i += 1

ax.plot(Plant_popt, "--", label="Theoretical plant population")
ax.set_xlabel(r"Generation ($n$)")
ax.set_ylabel(r"Plant population ($x$)")
ax.set_title(r"Plant population per generation for {0} simulations".format(sims+1))
plt.legend()
plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

## Question 2: Whooping crane population

Plot the crane population

In [5]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.plot(WC["Year"], WC["Cranes"])
ax.scatter(WC["Year"], WC["Cranes"])
ax.set_ylabel("Population")
ax.set_xlabel("Year")
ax.set_title("Whopping crane population from 1940 to 2015")

plt.grid()
plt.show()

<Figure size 720x288 with 1 Axes>

According to the site, Birds of North America$^{[1]}$, here's what we know about whopping cranes: 

> Monogamous and mate for life, won't find other mate unless mate dies

> Lays 1 to 3 eggs annually, though seldom fledge more than 1 young

> Starts producing eggs at 3 to 4 years old 

> Average lifespan of wild whopping crane is 25 years, although they can reach over 40 years old in captivity 

Before we model the whopping crane population, we assume:
> The population is independent of area (ie. It's the total whooping crane population in the world)

> There is no resource restrictions (ie. Unlimited food and stuff to build nests)

> The starting age distribution of the whopping crane is a Maxwell-Boltzmann distribution with an average of 25 years

>> This probablity distribution was choosen because of the low birth rate. I assumed that the low birth rate of the crane will cause it's age distribution to to be similar to the age distribution of countries with low birth rates, which resembles a Maxwell-Boltzman distribution.

>> Note that I will apply this only to the initial population, as births and deaths of cranes will cause the age distribution to deviate from this distribution.

### Using the Maxwell-Boltzmann distribution to find the percentage of reproductive pairs

The Maxwell-Boltzmann distribution:

$$M(x) = \sqrt{\frac{2}{\pi}}  \frac{x^2}{a} e^{-\frac{x^2}{2a^2}}$$

> Average: 

$$\mu = 2 a \sqrt{\frac{2}{\pi}}$$

> Setting our average to be 25 ($\mu = 25$):
    
$$a = \left( \frac{25}{2}\right) \sqrt{\frac{\pi}{2}} \approx 15.666$$

> As the cranes become fertile at around 3 to 4 years, we take the intergal of the population from 3.5 to $\infty$ to find the percentage of fertile pairs

In [17]:
def a(mu):
    a = (mu/2) * (pi/2)**0.5
    return a

def MB(x, mu):
    A = a(mu)
    M = (2/pi)**0.5 * (x**2/A) * exp(-x**2 / (2*A**2))
    return M

In [25]:
Ratio = quad(lambda x: MB(x, 25), 3.5, np.inf)[0]/quad(lambda x: MB(x, 25), 0, 60)[0]
print(Ratio)
x_s = np.arange(0, 100, 0.01)
plt.plot(x_s, MB(x_s, 25)/quad(lambda x: MB(x, 25), 0, np.inf)[0])

0.9992005016971202


<Figure size 432x288 with 1 Axes>

### Setting up the model

The whopping crane popultion (x) is determined by the diffrence relation:
$$x_{n+1} = [1 + B(x_n) - D(x_n)] x_n$$

$$\text{Where} \left\{ \begin{array}{ll}
            B(x_n) & \text{is a function determining births based on current population} \\
            D(x_n) & \text{is a function determining deaths based on current population}
            \end{array} \right.$$    

## Referances: 
[1] https://birdsna.org/Species-Account/bna/species/whocra/introduction